In [1]:
"""
FILE 1: BOND NAMES LIST file
Output: List of bonds that are (1) in TRACE and FISD (TRACE_mergent_names)
(2) PERMCO match via the crsp.msenames file, 
(3) bond-level filters (e.g., bond type, not variable coupon, etc.)

"""

### Imports Libraries ### 
import pandas as pd
import numpy as np
import wrds
from datetime import datetime, timedelta
import datetime
import sqlite3                   # for SQL query / merging purposes 

### Preliminary Variables/Paths Defined ### 
onedrive_pth = "C://Users/clj585/OneDrive - Northwestern University/"
download_dir = onedrive_pth + "(draft) adapted_code_bonds_python/"

# FISD List of variables
fisd_vars1 = ['issue_id', 'complete_cusip', 'offering_date', 'maturity', 'first_interest_date', 
              'dated_date', 'prospectus_issuer_name', 'bond_type', 'issuer_id', 'currency',
              'offering_amt', 'principal_amt', 'rule_144a', 'yankee',
              'coupon_type', 'coupon interest_frequency', 'pay_in_kind']
fisd_vars2 = ['asset_backed', 'convertible', 'exchangeable', 'defaulted', 'defeased', 
              'defeased_date', 'perpetual', 'preferred_security', 'putable', 
              'redeemable', 'security_level', 'security_pledge', 'slob', 
              'coupon_change_indicator']
collist = fisd_vars1 + fisd_vars2

In [12]:
"""
Step 0: 
Start with FISD list (requiring cusip9 also appears in TRACE data)

"""

### Connect to WRDS ###
conn = wrds.Connection()

# Get table FISD
bonds = conn.get_table(library='fisd', table='fisd_mergedissue', 
                       columns = collist)

# Drop any records missing primary IDs (full CUSIP and Issue ID)
bonds = bonds[(pd.notnull(bonds["issue_id"])==True) & 
              (pd.notnull(bonds['complete_cusip'])==True)]     # just in case - not necessary! 

# Rename one column 
colmap = {'complete_cusip' : "cusip9"}
bonds = bonds.rename(columns=colmap)

# Get TRACE Enhanced Table From WRDS
trace = conn.get_table(library='trace', table='trace_enhanced_names')
#trace = trace[trace['cusip_id'].isna() == False]

Enter your WRDS username [clj585]:cindylu
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


In [8]:
bonds

,issue_id,cusip9,offering_date,maturity,first_interest_date,dated_date,prospectus_issuer_name,bond_type,issuer_id,currency,...,defeased,defeased_date,perpetual,preferred_security,putable,redeemable,security_level,security_pledge,slob,coupon_change_indicator
0,1.0,000361AA3,1989-10-24,2001-11-01,1990-05-01,1989-11-01,AAR CORP,CDEB,3.0,None,...,N,None,N,N,N,N,SEN,None,N,N
1,2.0,000361AB1,1993-10-12,2003-10-15,1994-04-15,1993-10-15,AAR CORP,CDEB,3.0,None,...,N,None,N,N,N,N,SEN,None,N,N
2,3.0,00077DAB5,1994-01-07,1996-01-12,None,1994-01-14,ABN AMRO BK N V N Y BRH,CMTN,40263.0,None,...,N,None,N,N,N,N,SEN,None,N,N
3,4.0,00077DAF6,1994-07-27,2009-08-01,1995-02-01,1994-08-02,ABN AMRO BK N V N Y BRH,USBN,40263.0,None,...,N,None,N,N,N,Y,SENS,None,N,N
4,5.0,00077TAA2,1993-05-20,2023-05-15,1993-11-15,1993-05-15,ABN AMRO BK N V N Y BRH,CDEB,40263.0,None,...,N,None,N,N,N,Y,SUB,None,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,994956.0,40057J3F1,2021-08-31,2026-09-03,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,None,...,N,None,N,N,N,N,SEN,None,N,N
28299,994957.0,40057J3H7,2021-08-31,2026-09-03,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,None,...,N,None,N,N,N,N,SEN,None,N,N
28300,994958.0,40057J3T1,2021-08-31,2023-09-06,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,None,...,N,None,N,N,N,N,SEN,None,N,N
28301,994959.0,40057J3Z7,2021-08-31,2023-09-06,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,None,...,N,None,N,N,N,N,SEN,None,N,N


In [9]:
trace

,cusip_id,bond_sym_id,company_symbol,bloomberg_identifier,st_date,end_date
0,None,AAPY4868409,AAPY,None,2019-08-05,2019-08-05
1,None,ABDN3973417,ABDN,None,2013-02-22,2013-02-28
2,None,ACCX.AP,ACCX,None,2016-09-26,2016-09-26
3,None,ACCX.AS,ACCX,None,2016-06-09,2016-06-09
4,None,ACCX.AW,ACCX,None,2012-11-26,2012-11-26
...,...,...,...,...,...,...
361157,Y9T11KBZ7,UOVE4340990,UOVE,None,2016-02-26,2017-02-24
361158,Y9T36FAD9,WHARF3998558,WHARF,None,2013-04-22,2017-01-05
361159,ZR7955878,None,None,None,2019-09-25,2019-09-25
361160,ZS1854935,CHCJF4829082,CHCJF,None,2019-04-25,2019-04-25


In [13]:
### 1st Merge Step: Require Match in TRACE File ### 
"""
* 2. Require match in TRACE file (implying it was traded after 2002);
proc sql; 
	create table bonds as
	select distinct l.cusip9, l.*
	from bonds as l  
	left join trace.trace_enhanced_names (where=(not cmiss(cusip_id) and cusip_id^='') ) as r
	on l.cusip9=r.cusip_id
	where not missing(r.cusip_id)
;quit;
"""

con = sqlite3.connect("fisd.db")
bonds.to_sql("bonds", con, index=False, if_exists='replace')
trace.to_sql("trace", con, index=False, if_exists='replace')

sql = """SELECT DISTINCT a.cusip9, a.*, trace2.cusip_id
        FROM bonds as a
       LEFT JOIN
       (SELECT * FROM trace
        where cusip_id != "" or cusip_id != "None"
       ) AS trace2 
       ON a.cusip9 = trace2.cusip_id 
       where (trace2.cusip_id != "" or trace2.cusip_id != "None");""" 

# 215260 rows should be 
bonds = pd.read_sql_query(sql, con)
bonds

,cusip9,issue_id,cusip9,offering_date,maturity,first_interest_date,dated_date,prospectus_issuer_name,bond_type,issuer_id,...,defeased_date,perpetual,preferred_security,putable,redeemable,security_level,security_pledge,slob,coupon_change_indicator,cusip_id
0,000361AB1,2.0,000361AB1,1993-10-12,2003-10-15,1994-04-15,1993-10-15,AAR CORP,CDEB,3.0,...,None,N,N,N,N,SEN,None,N,N,000361AB1
1,00077TAA2,5.0,00077TAA2,1993-05-20,2023-05-15,1993-11-15,1993-05-15,ABN AMRO BK N V N Y BRH,CDEB,40263.0,...,None,N,N,N,Y,SUB,None,N,N,00077TAA2
2,00077TAB0,6.0,00077TAB0,1993-10-13,2093-10-15,1994-04-15,1993-10-15,ABN AMRO BK N V N Y BRH,CDEB,40263.0,...,None,N,N,N,Y,SUB,None,N,N,00077TAB0
3,001765AC0,38.0,001765AC0,1986-09-11,2016-09-15,1987-03-15,1986-09-15,AMR CORP DEL,CDEB,20.0,...,None,N,N,N,N,SEN,None,N,N,001765AC0
4,001765AE6,40.0,001765AE6,1990-03-07,2020-03-15,1990-09-15,1990-03-15,AMR CORP DEL,CDEB,20.0,...,None,N,N,N,N,SEN,None,N,N,001765AE6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215255,40057J3F1,994956.0,40057J3F1,2021-08-31,2026-09-03,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,...,None,N,N,N,N,SEN,None,N,N,40057J3F1
215256,40057J3H7,994957.0,40057J3H7,2021-08-31,2026-09-03,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,...,None,N,N,N,N,SEN,None,N,N,40057J3H7
215257,40057J3T1,994958.0,40057J3T1,2021-08-31,2023-09-06,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,...,None,N,N,N,N,SEN,None,N,N,40057J3T1
215258,40057J3Z7,994959.0,40057J3Z7,2021-08-31,2023-09-06,None,2021-09-03,GS FIN CORP,CMTZ,40163.0,...,None,N,N,N,N,SEN,None,N,N,40057J3Z7


In [ ]:
con = sqlite3.connect("fisd.db")
trace.to_sql("trace", con, index=False, if_exists='replace')

testsql = """SELECT * FROM trace
        where cusip_id != "" or cusip_id != "None"
 """
blah = pd.read_sql_query(testsql, con)
blah

In [ ]:
trace

In [ ]:
trace = conn.get_table(library='trace', table='trace_enhanced_names')

In [ ]:
trace[trace['cusip_id'].isna() == False]

In [ ]:



"""*--------------------- Maturity>=2022, CURRENCY AND DOMESTIC, $1M+ ------------------;
data bonds1; 
	set bonds; 
	if currency='USD' or missing(currency) ; 
	if yankee^='Y';															/*Yankee = A flag indicating that the issue has been issued by a foreign issuer, but has been registered with the SEC and is payable in dollars.;*/
	if offering_amt>1000;													/* >1million. Offering_amt = The par value of debt initially issued.;*/
	if (maturity >= '01Jan2012'd) and (offering_date <= '31Dec2020'd);		/* Note: only perpetual bonds have missing maturity*/
run;

*--------------------- BOND TYPE ------------------;
data bonds1; 
	set bonds1;
	if rule_144a^='Y'   ; 														/* Kick 144A bonds. Rule_144a= A flag denoting that the issue is a private placement exempt from registration under SEC Rule 144a. Rule 144a issues are generally offered to a limited number of institutional investors;*/
	if bond_type in ('CMTN' 'CMTZ' 'CDEB' 'RNT' 'CZ' )   ;						/* bond_type in  CMTN [US Corporate MTN], CMTZ [US Corporate MTN Zero], CDEB [US Corporate Debentures], RNT [Retail Note], CZ [US Corporate Zero]) */
	if interest_frequency in ('2' '0' '4' '12' '1')   ;							/* More than 99% have this*/
	if coupon_type^='V'   ;														/* Kick variable rate bonds. Are these floating bonds? Their coupon changes with some interest rate (e.g., LIBOR). Not standardized how often and frequently coupon changes. */
	if perpetual^='Y'   ;														/* Kick perpetual */
	if convertible^='Y'   ; 													/* Kick convertible bonds. Convertible = Flag indicating the issue can be converted to the common stock (or other security) of the issuer. Further information can be found in the CONVERTIBLE table.;*/
	if exchangeable ^= 'Y'   ;													/* Kick exchangeable bonds. Can be exchanged for securities of other firms (usually subsidiary) */
	if putable ^= 'Y'   ;														/* Kick putable bonds. They have a 'put option' forcing early repayment */
	if pay_in_kind^='Y'; 														/* Kick pay_in_kind: means that interest can be paid by giving more of the same bond issue or by giving other securities instead of cash. .;*/	
run;

*--------------------- SECURED OR ASSET-BACKED? ------------------;
data bonds1; 
	set bonds1;
	if preferred_security ^='Y'   ; 											/* Kick preferred_securities. Preferred_security = Flag indicating this issue is a preferred security;*/
	if security_level^=SS   ; 													/* SS= senior secured. We skip it because some are in the Hotchkiss treatment sample.*/
	if asset_backed^='Y'   ; 													/* Kick asset_backed bonds. Asset_backed = Flag indicating that the issue is an asset-backed issue, that is collateralized by a portfolio of loans or assets other than single family mortgages. */
	if Slob^='Y'   ; 															/* if Slob^=Y. Slob = flag denoting that the issue is a secured lease obligation issue (i.e., an issue secured by one or more leases issued in a sales leaseback transaction by an electric utility). Kick security_pledge bonds. */
	if security_pledge^='M'   ; 												/* Kick security_pledge bonds. Security_pledge = A flag indicating that certain assets have been pledged as security for the issue. Can be missing, M [mortgate], or N [Note].*/
run;

*--------------------- DATA AVAILABILITY ------------------;
data bonds1; 
	set bonds1;
	if not cmiss(offering_date,maturity);
	if cmiss(coupon,first_interest_date, dated_date) and interest_frequency^='0' then delete;	/* require coupon-variables UNLESS zero bond */
run;

*--------------------- INDUSTRY CODE: Kick banks ------------------;
** Include industry_type;
proc sql;
	create table bonds1 as
	select distinct l.*, r.Industry_group, r.industry_code
	from bonds1 as l
	left join fisd.fisd_issuer as r
	on l.issuer_id=r.issuer_id
	having r.industry_code not in ("20")  			/* 20=banks */
;quit;
"""

In [ ]:
""" SAS CODE TO ADAPT

*-----------------------------------------------
* STEP 0: start with FISD cusip9s (require that cusip is also in TRACE file);
* STEP 1: Place bond-level filters based on FISD variables;
* STEP 2: Include Permco match from CRSP.MSENAMES file (via ncusip6-permco);
*******************************************************************************/

** Load libraries, macros, labels;
*%include "~/sasuser.v94/codetocheck/0_include_libraries_macros_labels.sas";*"\\Mac\Home\Desktop\code bond returns\macros\0_include_libraries_macros_labels.sas";




*==============================================================================;
*;
* Step 0: Start with FISD list (requiring cusip9 also appears in TRACE data);
*;
*==============================================================================;
%let fisd_vars1 = issue_id complete_cusip offering_date maturity first_interest_date dated_date prospectus_issuer_name bond_type issuer_id currency  offering_amt  principal_amt rule_144a yankee coupon_type coupon interest_frequency pay_in_kind;
%let fisd_vars2 = asset_backed convertible exchangeable defaulted defeased defeased_date perpetual preferred_security putable redeemable  security_level security_pledge slob coupon_change_indicator;

* 1. FISD data;
data bonds;
	set fisd.fisd_mergedissue;
	where (not cmiss(complete_cusip,issue_id)) and (complete_cusip^='');
	keep &fisd_vars1. &fisd_vars2;
	rename complete_cusip=cusip9;
run;

* 2. Require match in TRACE file (implying it was traded after 2002);
proc sql; 
	create table bonds as
	select distinct l.cusip9, l.*
	from bonds as l  
	left join trace.trace_enhanced_names (where=(not cmiss(cusip_id) and cusip_id^='') ) as r
	on l.cusip9=r.cusip_id
	where not missing(r.cusip_id)
;quit;



*==============================================================================;
*;
* Step 1: Filter at Bond Level. Also, create record of how N_obs changes as a result of doing so;
*;
*==============================================================================;

*--------------------- Maturity>=2022, CURRENCY AND DOMESTIC, $1M+ ------------------;
data bonds1; 
	set bonds; 
	if currency='USD' or missing(currency) ; 
	if yankee^='Y';															/*Yankee = A flag indicating that the issue has been issued by a foreign issuer, but has been registered with the SEC and is payable in dollars.;*/
	if offering_amt>1000;													/* >1million. Offering_amt = The par value of debt initially issued.;*/
	if (maturity >= '01Jan2012'd) and (offering_date <= '31Dec2020'd);		/* Note: only perpetual bonds have missing maturity*/
run;

*--------------------- BOND TYPE ------------------;
data bonds1; 
	set bonds1;
	if rule_144a^='Y'   ; 														/* Kick 144A bonds. Rule_144a= A flag denoting that the issue is a private placement exempt from registration under SEC Rule 144a. Rule 144a issues are generally offered to a limited number of institutional investors;*/
	if bond_type in ('CMTN' 'CMTZ' 'CDEB' 'RNT' 'CZ' )   ;						/* bond_type in  CMTN [US Corporate MTN], CMTZ [US Corporate MTN Zero], CDEB [US Corporate Debentures], RNT [Retail Note], CZ [US Corporate Zero]) */
	if interest_frequency in ('2' '0' '4' '12' '1')   ;							/* More than 99% have this*/
	if coupon_type^='V'   ;														/* Kick variable rate bonds. Are these floating bonds? Their coupon changes with some interest rate (e.g., LIBOR). Not standardized how often and frequently coupon changes. */
	if perpetual^='Y'   ;														/* Kick perpetual */
	if convertible^='Y'   ; 													/* Kick convertible bonds. Convertible = Flag indicating the issue can be converted to the common stock (or other security) of the issuer. Further information can be found in the CONVERTIBLE table.;*/
	if exchangeable ^= 'Y'   ;													/* Kick exchangeable bonds. Can be exchanged for securities of other firms (usually subsidiary) */
	if putable ^= 'Y'   ;														/* Kick putable bonds. They have a 'put option' forcing early repayment */
	if pay_in_kind^='Y'; 														/* Kick pay_in_kind: means that interest can be paid by giving more of the same bond issue or by giving other securities instead of cash. .;*/	
run;

*--------------------- SECURED OR ASSET-BACKED? ------------------;
data bonds1; 
	set bonds1;
	if preferred_security ^='Y'   ; 											/* Kick preferred_securities. Preferred_security = Flag indicating this issue is a preferred security;*/
	if security_level^=SS   ; 													/* SS= senior secured. We skip it because some are in the Hotchkiss treatment sample.*/
	if asset_backed^='Y'   ; 													/* Kick asset_backed bonds. Asset_backed = Flag indicating that the issue is an asset-backed issue, that is collateralized by a portfolio of loans or assets other than single family mortgages. */
	if Slob^='Y'   ; 															/* if Slob^=Y. Slob = flag denoting that the issue is a secured lease obligation issue (i.e., an issue secured by one or more leases issued in a sales leaseback transaction by an electric utility). Kick security_pledge bonds. */
	if security_pledge^='M'   ; 												/* Kick security_pledge bonds. Security_pledge = A flag indicating that certain assets have been pledged as security for the issue. Can be missing, M [mortgate], or N [Note].*/
run;

*--------------------- DATA AVAILABILITY ------------------;
data bonds1; 
	set bonds1;
	if not cmiss(offering_date,maturity);
	if cmiss(coupon,first_interest_date, dated_date) and interest_frequency^='0' then delete;	/* require coupon-variables UNLESS zero bond */
run;

*--------------------- INDUSTRY CODE: Kick banks ------------------;
** Include industry_type;
proc sql;
	create table bonds1 as
	select distinct l.*, r.Industry_group, r.industry_code
	from bonds1 as l
	left join fisd.fisd_issuer as r
	on l.issuer_id=r.issuer_id
	having r.industry_code not in ("20")  			/* 20=banks */
;quit;


*==============================================================================;
*;
* Step 2: Include Permco from crsp.msenames on ncusip6, which contains {ncusip6,permco}-combinations;
*;
*==============================================================================;
proc sql;
	create table bonds2 as
	select distinct l.cusip9, r.permco, l.*	
					, ( min(r.namedt) <= l.offering_date <= max(r.nameendt) ) as d_offering_covered
					, (not missing(permco)) as d_permco
						/* , l.issue_id , l.maturity, l.prospectus_issuer_name
						, min(r.namedt) as start_cusip6_permco
						, max(r.nameendt) as end_cusip6_permco */
	from bonds1 as l
	left join crsp.msenames as r
	on substr(l.cusip9,1,6)=substr(r.ncusip,1,6)
	group by substr(r.ncusip,1,6) , permco					/* for each cusip6-permco */
	/*having ( not cmiss(l.cusip9,permco))*/
;quit;

	

*==============================================================================;
*;
* SAVE;
*;
*==============================================================================;
%let data_out = bond.bond_list_permco; libname bond "~/sasuser.v94/codetocheck/bond";
data &data_out.;
	set bonds2;
run;

"""